In [34]:
import pandas as pd

In [35]:
df = pd.read_csv("Datasets/movies.csv", encoding='latin1')



In [44]:
df.sample(6)

,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
371,2001,177.00000,"Action, Thriller",7.2,35.0,Suresh Krishna,Kamal Haasan,Raveena Tandon,Manisha Koirala
14198,2019,111.00000,"Crime, Mystery, Thriller",8.7,85.0,Rohit Arora,Saasha Aery,Sidharth Bhardwaj,Anahita Bhooshan
1600,2018,124.00000,"Comedy, Drama",8.0,35.0,Amit Ravindernath Sharma,Ayushmann Khurrana,Neena Gupta,Gajraj Rao
394,1964,132.29448,"Action, Adventure, Romance",6.0,8.0,Mohammed Hussain,Dara Singh,Mumtaz,Kamran
10141,1976,132.29448,"Action, Crime, Drama",5.4,32.0,Raj Khosla,Sunil Dutt,Saira Banu,Vinod Khanna
2370,1960,132.29448,Drama,5.7,12.0,R. Krishnan,S. Panju,Padmini,Balraj Sahni


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7919 entries, 1 to 15508
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      7919 non-null   int64  
 1   Duration  7919 non-null   float64
 2   Genre     7919 non-null   object 
 3   Rating    7919 non-null   float64
 4   Votes     7919 non-null   float64
 5   Director  7919 non-null   object 
 6   Actor 1   7919 non-null   object 
 7   Actor 2   7919 non-null   object 
 8   Actor 3   7919 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 618.7+ KB


In [38]:
df.dropna(subset=['Rating'], inplace=True)

In [39]:
# df['Votes'] = df['Votes'].str.replace(',', '', regex=False)
df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')

In [40]:
df['Year'] = df['Year'].astype(str)           
df['Year'] = df['Year'].str.extract(r'(\d{4})') 
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')


In [41]:
df['Duration'] = df['Duration'].astype(str)
df['Duration'] = df['Duration'].str.replace(',', '', regex=False)
df['Duration'] = df['Duration'].str.extract(r'(\d+)')  
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')


In [42]:
df['Duration'].fillna(df['Duration'].mean(), inplace=True)
df['Votes'].fillna(df['Votes'].median(), inplace=True)
for col in ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']:
    df[col].fillna('Unknown', inplace=True)

/var/folders/ww/mt9vwdt90_5_tqhykvt71gdc0000gn/T/ipykernel_1062/858332472.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Duration'].fillna(df['Duration'].mean(), inplace=True)
/var/folders/ww/mt9vwdt90_5_tqhykvt71gdc0000gn/T/ipykernel_1062/858332472.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

In [43]:
df.drop(columns=['Name'], inplace=True)

In [46]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Rating'])
y = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [55]:
from category_encoders import TargetEncoder
cat_cols = ['Director', 'Actor 1', 'Actor 2', 'Actor 3']
encoder = TargetEncoder(cols=cat_cols)
X_train[cat_cols] = encoder.fit_transform(X_train[cat_cols], y_train)
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

In [56]:
X_train['Genre'] = X_train['Genre'].apply(lambda x: [g.strip() for g in x.split(',')] if isinstance(x, str) else [])
X_test['Genre'] = X_test['Genre'].apply(lambda x: [g.strip() for g in x.split(',')] if isinstance(x, str) else [])

In [57]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_genres = pd.DataFrame(mlb.fit_transform(X_train['Genre']),
                            columns=mlb.classes_,
                            index=X_train.index)
test_genres = pd.DataFrame(mlb.transform(X_test['Genre']),
                           columns=mlb.classes_,
                           index=X_test.index)
X_train = pd.concat([X_train.drop(columns='Genre'), train_genres], axis=1)
X_test = pd.concat([X_test.drop(columns='Genre'), test_genres], axis=1)

In [58]:
print(X_train.dtypes)
print(X_test.dtypes)

Year             int64
Duration       float64
Votes          float64
Director       float64
Actor 1        float64
Actor 2        float64
Actor 3        float64
Action           int64
Adventure        int64
Animation        int64
Biography        int64
Comedy           int64
Crime            int64
Documentary      int64
Drama            int64
Family           int64
Fantasy          int64
History          int64
Horror           int64
Music            int64
Musical          int64
Mystery          int64
News             int64
Romance          int64
Sci-Fi           int64
Sport            int64
Thriller         int64
Unknown          int64
War              int64
Western          int64
dtype: object
Year             int64
Duration       float64
Votes          float64
Director       float64
Actor 1        float64
Actor 2        float64
Actor 3        float64
Action           int64
Adventure        int64
Animation        int64
Biography        int64
Comedy           int64
Crime            int

In [62]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=500,
    max_depth=15,
    min_samples_leaf=3,
    random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)


In [63]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

preds = model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, preds)

print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")


MAE: 0.930
RMSE: 1.207
R²: 0.216
